**Setup Ollama**

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve &
!ollama pull qwen3:4b
!pip install -q mcp-server sentence-transformers  PyMuPDF faiss-cpu
!pip install -q gradio
!pip install -q langchain-community
!pip install -U --quiet langgraph
!pip install -q python-docx
!pip install -q chromadb

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
nohup: appending output to 'nohup.out'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 43.1 MB

In [2]:
!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 99.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 94.0 MB/s eta 0:00:00


In [3]:
from __future__ import annotations

import os
import re
import uuid
import json
import queue
import threading
import tempfile
from pathlib import Path
from typing import List, Dict, Tuple, Any

import gradio as gr
import pdfplumber
import docx
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import requests

In [ ]:
# from langchain_community.embeddings import HuggingFaceEmbeddings
# from langchain_community.vectorstores import Chroma
# from langchain_core.documents import Document

# # Use a BGE embedding model from HuggingFace
# bge_embedding = HuggingFaceEmbeddings(
#     model_name="BAAI/bge-m3",
#     encode_kwargs={"normalize_embeddings": True}
# )

# vectorstore = Chroma.from_documents(doc_splits, bge_embedding, persist_directory="./chroma_db")
# retriever = vectorstore.as_retriever()

In [ ]:
# from langchain.tools.retriever import create_retriever_tool

# retriever_tool = create_retriever_tool(
#     retriever,
#     name="search_user_documents",
#     description="Searches the user's uploaded documents and returns the most relevant passages with page and paragraph and document info.",
# )

In [ ]:
# retriever_tool.invoke({"query": "what is LIME ?"})

'needtoevaluatethemodelasawholebeforedeployingit“in\nprediction into a trustworthy one.\nthe wild”. To make this decision, users need to be confident\nIn this work, we propose LIME, a novel explanation tech-\nthatthemodelwillperformwellonreal-worlddata,according\nnique that explains the predictions of any classifier in an in-\n\nIn this work, we propose LIME, a novel explanation tech-\nthatthemodelwillperformwellonreal-worlddata,according\nnique that explains the predictions of any classifier in an in-\nto the metrics of interest. Currently, models are evaluated\nterpretableandfaithfulmanner,bylearninganinterpretable\nusing accuracy metrics on an available validation dataset.\nmodel locally around the prediction. We also propose a\n\nbeing explained. LIME samples instances, gets pre- selecting an appropriate family of explanations from a set of\ndictions using f, and weighs them by the proximity multiple interpretable model classes, thus adapting to the\nto the instance being explained

In [ ]:
# from langchain_community.llms.ollama import Ollama
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# # model = Ollama(model="Qwen/Qwen1.5-14B-Chat-GPTQ-Int4")
# # MODEL_NAME = "Qwen/Qwen1.5-14B-Chat-GPTQ-Int4"

# # tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# # model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto", trust_remote_code=True)

# # pipe = pipeline(
# #     "text-generation",
# #     model=model,
# #     tokenizer=tokenizer,
# #     max_new_tokens=1024,
# #     temperature=0,
# #     do_sample=True,
# # )

# # response_model = Ollama(model="Qwen/Qwen1.5-14B-Chat-GPTQ-Int4")
# response_model = Ollama(model="qwen3:4b")

# def generate_query_or_respond(state):
#     user_message = state["messages"][-1]["content"]
#     # 1. Run retriever/tool for context
#     retrieved_docs = retriever_tool.run(user_message)  # Or retriever.get_relevant_documents(...)
#     context = "\n\n".join([
#         doc.page_content if hasattr(doc, "page_content") else str(doc)
#         for doc in retrieved_docs
#     ])
#     # 2. Compose prompt for LLM
#     prompt = f"Use only the following context to answer:\n{context}\n\nUser question: {user_message}\nAnswer:"
#     # 3. Get LLM response
#     answer = response_model.invoke(prompt)
#     # 4. Return as a new message
#     return {"messages": state["messages"] + [{"role": "assistant", "content": answer}]}


In [ ]:
# input = {
#     "messages": [
#         {
#             "role": "user",
#             "content": "What does the paper talk about?",
#         }
#     ]
# }
# output = generate_query_or_respond(input)


In [ ]:
# print(output["messages"][-1]["content"])

<think>
Okay, let's try to figure out what the paper is about based on the given text. The user provided a lot of text that seems to be a mix of different parts, possibly from a paper or article. Let me start by breaking down the information.

First, there's an ISBN number: ISBN978-1-4503-4232-2/16/08...$15.00. That's probably the book's ISBN, but the text might be a chapter or section from that book. Then there's a DOI: http://dx.doi.org/10.1145/2939672.2939778. That's a Digital Object Identifier, which links to an academic paper. The title mentioned is "Explaining Predictions is as Important as Accuracy for Machine Learning" by authors like M. T. Dzindolete, S. A. Petersen, R. A. Pomranky, L. G. Pierce, and H. P. Beck. 

The paper seems to discuss the importance of explainability in machine learning models. The key points mentioned are that explaining predictions is as crucial as achieving high accuracy. The authors argue that understanding why a model makes certain predictions is vi

**Handle file**

In [4]:
from pathlib import Path

def get_file_bytes_and_name(pdf_file):
    print("DEBUG: pdf_file type:", type(pdf_file))
    print("DEBUG: pdf_file dir:", dir(pdf_file))
    print("DEBUG: pdf_file repr:", repr(pdf_file))
    # Standard file-like object (e.g. Python's open, script mode)
    if hasattr(pdf_file, "read"):
         return pdf_file.read(), Path(pdf_file.name).name
    if isinstance(pdf_file, str):
        file_path = Path(pdf_file)
        with open(file_path, "rb") as f:
            return f.read(), file_path.name
    # else:
    #     raise ValueError("Could not extract file bytes from uploaded file.")
    raise ValueError("Could not extract file bytes from uploaded file.")


In [5]:
from langchain.schema import Document
import pdfplumber

def pdf_to_documents(pdf_path: str) -> list[Document]:
    docs = []
    with pdfplumber.open(pdf_path) as pdf:
        for page_no, page in enumerate(pdf.pages, start=1):
            text = page.extract_text() or ""
            # simple paragraph split – tweak as you like
            for para_no, para in enumerate(text.split("\n\n"), start=1):
                cleaned = para.strip()
                if cleaned:           # skip blank chunks
                    docs.append(
                        Document(
                            page_content=cleaned,
                            metadata={"page": page_no, "paragraph": para_no}
                        )
                    )
    return docs


In [6]:
def format_sources(retrieved_docs):
    numbered, exposed = [], []
    for i, doc in enumerate(retrieved_docs, start=1):
        p, n = doc.metadata.get("page"), doc.metadata.get("paragraph")
        txt   = doc.page_content
        numbered.append(f"({i}) [page {p} ¶{n}] {txt}")
        exposed.append(f"[page {p} ¶{n}] {txt}")
    return "\n".join(numbered), exposed

**Create Vector database**

In [7]:
def build_vectorstore_from_pdf(pdf_file, embedding_model, persist_directory=None):
    import tempfile
    from pathlib import Path
    import pdfplumber
    from langchain.docstore.document import Document
    from langchain_text_splitters import RecursiveCharacterTextSplitter
    from langchain_community.vectorstores import Chroma
    import re

    # --- read bytes & filename ---
    file_bytes, file_name = get_file_bytes_and_name(pdf_file)

    # --- write to a temp file ---
    temp_path = Path(tempfile.gettempdir()) / file_name
    with open(temp_path, "wb") as f:
        f.write(file_bytes)

        # --- NEW: create one-paragraph Documents with location metadata ---
    docs_list = pdf_to_documents(str(temp_path))      # ← replaces the old pdfplumber loop

    # (optional) keep your existing splitter so long paragraphs still get trimmed
    splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=700, chunk_overlap=100
    )
    doc_splits = splitter.split_documents(docs_list)  # metadata is copied to every chunk

    # --- build the Chroma vector store exactly as before ---
    vectordb = Chroma.from_documents(
        doc_splits,
        embedding_model,
        persist_directory=persist_directory,
    )
    return vectordb


**Create tool**

In [10]:
def build_retriever_tool(vectorstore, name="search_user_documents"):
    retriever = vectorstore.as_retriever()
    from langchain.tools.retriever import create_retriever_tool
    retriever_tool = create_retriever_tool(
        retriever,
        name=name,
        description="Searches uploaded documents and returns relevant passages."
    )
    return retriever_tool


In [12]:
# retrieved, _ = retriever.similarity_search_with_score(query, k=3)
# numbered_block, exposed_sources = format_sources([r[0] for r in retrieved])
retrieved = retriever_tool.invoke(query)
numbered_block, exposed_sources = format_sources(retrieved)

NameError: name 'retriever_tool' is not defined

In [13]:
!ollama pull qwen3:4b

In [ ]:
!nohup ollama serve &

nohup: appending output to 'nohup.out'


In [16]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms.ollama import Ollama
import traceback
import torch

# Check if CUDA is available and set the device accordingly
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

embedding_model = HuggingFaceEmbeddings(
    model_name="BAAI/bge-m3",
    encode_kwargs={"normalize_embeddings": True, "device": device} # Specify device
)
response_model = Ollama(model="qwen3:4b")

def pdf_qa_pipeline(pdf_file, query_text, k=3):
    try:
        vectorstore  = build_vectorstore_from_pdf(pdf_file, embedding_model)

        #  Get real Document objects (with metadata) straight from Chroma
        retriever     = vectorstore.as_retriever(search_kwargs={"k": k})
        retrieved_docs = retriever.get_relevant_documents(query_text)
        # └── each item here is a Document, so .metadata works

        if not retrieved_docs:
            return "No relevant information found in the uploaded document."

        numbered_block, exposed_sources = format_sources(retrieved_docs)

        prompt = f"""You are a helpful assistant that must answer **only** from the numbered EXCERPTS below.
If the answer is not present, reply exactly: "Not found in the document."

EXCERPTS:
{numbered_block}

QUESTION: {query_text}
ANSWER:"""

        answer = response_model.invoke(prompt).strip()
        return f"{answer}\n\n**Paragraphs used**\n" + "\n".join(exposed_sources)

    except Exception as e:
        traceback.print_exc()
        return f"An error occurred: {str(e)}"



# def pdf_qa_pipeline(pdf_file, query_text):

#     try:
#         print("Starting pipeline...")
#         vectorstore = build_vectorstore_from_pdf(pdf_file, embedding_model)
#         print("Vectorstore built.")
#         retriever_tool = build_retriever_tool(vectorstore)
#         print("Retriever tool built.")
#         retrieved_docs = retriever_tool.run(query_text)
#         print("Retrieved docs:", retrieved_docs)
#         context = "\n\n".join([
#             doc.page_content if hasattr(doc, "page_content") else str(doc)
#             for doc in retrieved_docs
#         ])

#         # ---- 1. Print the context chunk you are sending to the LLM ----
#         print("CONTEXT PASSED TO LLM:", repr(context[:500]))

#         # ---- 2. Return a default answer if the context is empty ----
#         if not context.strip():
#             return "No relevant information found in the uploaded document."

#         # ---- 3. Use a stricter prompt ----
#         prompt = f"""Answer the user's question **using only** the information provided by the file uploaded by ther user.
#     Provide both the page and paragraph that you have found to support your response.
#     If the context does not contain the answer, say 'Not found in the document.'

#     Context:
#     {context}

#     Question: {query_text}
#     Answer:"""

#         answer = response_model.invoke(prompt)
#         return answer

#     except Exception as e:
#         print("ERROR OCCURRED IN pdf_qa_pipeline!")
#         traceback.print_exc()
#         return f"An error occurred: {str(e)}"

Using device: cuda


In [17]:
demo = gr.Interface(
    fn=pdf_qa_pipeline,
    inputs=[
        gr.File(label="Upload PDF", file_types=[".pdf"]),
        gr.Textbox(label="Question", placeholder="Ask anything about the PDF..."),
    ],
    outputs=gr.Textbox(label="Answer"),
    title="PDF Q&A - All In One",
    description="Upload a PDF and ask a question—get an LLM answer grounded only in your document."
)

if __name__ == "__main__":
    demo.launch(mcp_server=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1013159e50f12cd83b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)

🔨 MCP server (using SSE) running at: https://1013159e50f12cd83b.gradio.live/gradio_api/mcp/sse
